In [13]:
# import dataframe and check the data
import pandas as pd
bookings = pd.read_csv(
    '/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-mihajlova/shared/homeworks/python_ds_miniprojects/2/bookings.csv',
    sep = ';'
)

In [14]:
bookings.head()

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [15]:
bookings.shape

(119390, 21)

In [16]:
bookings.dtypes.value_counts()


int64      10
object     10
float64     1
dtype: int64

In [17]:
bookings.columns

Index(['Hotel', 'Is Canceled', 'Lead Time', 'arrival full date',
       'Arrival Date Year', 'Arrival Date Month', 'Arrival Date Week Number',
       'Arrival Date Day of Month', 'Stays in Weekend nights',
       'Stays in week nights', 'stays total nights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'Reserved Room Type', 'Assigned room type',
       'customer type', 'Reservation Status', 'Reservation status_date'],
      dtype='object')

In [18]:
# rename the columns to lower case and space-free
def change_names(name):
    return name.lower().replace(' ','_')
# use function instead of a dictionary
bookings = bookings.rename(columns = change_names)   

In [19]:
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [22]:
# choose top-5 countries (only non-calcelled orders)
bookings.query('is_canceled == 0')\
        .groupby('country')\
        .agg({'hotel':'count'})\
        .rename(columns = {'hotel':'orders_amnt'})\
        .sort_values('orders_amnt', ascending=False)\
        .head()


,orders_amnt
country,
PRT,21071
GBR,9676
FRA,8481
ESP,6391
DEU,6069


In [26]:
# avg days of bookings per hotel
bookings.groupby('hotel')\
        .agg({'stays_total_nights':'mean'})\
        .round(2)

,stays_total_nights
hotel,
City Hotel,2.98
Resort Hotel,4.32


In [31]:
# find the number of changing rooms
bookings.query("reserved_room_type != assigned_room_type").shape[0]

14917

In [33]:
# what is the most popular month in 2017? in 2016?
bookings.groupby(['arrival_date_year', 'arrival_date_month'])\
        .agg({'hotel':'count'})\
        .sort_values(['arrival_date_year','hotel'], ascending=False)

hotel
arrival_date_year arrival_date_month       
2017              May                  6313
                  April                5661
                  June                 5647
                  July                 5313
                  March                4970
                  August               4925
                  February             4177
                  January              3681
2016              October              6203
                  May                  5478
                  April                5428
                  September            5394
                  June                 5292
                  August               5063
                  March                4824
                  July                 4572
                  November             4454
                  February             3891
                  December             3860
                  January              2248
2015              September            5114
                  October              4957
                  August               3889
                  December             2920
                  July                 2776
                  November             2340

In [34]:
# what is the most popular month per year for cancellations?
bookings.query('is_canceled == 1')\
        .groupby(['arrival_date_year','arrival_date_month'])\
        .agg({'hotel':'count'})\
        .sort_values(['arrival_date_year','hotel'], ascending=False)

hotel
arrival_date_year arrival_date_month       
2017              May                  2762
                  April                2463
                  June                 2439
                  July                 1984
                  August               1816
                  March                1672
                  February             1359
                  January              1250
2016              October              2514
                  June                 2096
                  April                2061
                  September            2022
                  May                  1915
                  August               1825
                  November             1636
                  July                 1499
                  March                1477
                  December             1398
                  February             1337
                  January               557
2015              September            2094
                  October              1732
                  August               1598
                  July                 1259
                  December              973
                  November              486

In [35]:
bookings[['adults','children','babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [38]:
bookings['total_kids'] = bookings['children'] + bookings['babies']
bookings.groupby('hotel')\
    .agg({'total_kids':'mean'})\
    .round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


In [39]:
# check how kids relies on number of booking cancellations (churn rate)
bookings['has_kids'] = bookings['total_kids'] > 0

In [45]:
total_users = bookings.has_kids.value_counts()

In [46]:
cancel_users = bookings.query('is_canceled == 1').has_kids.value_counts()

In [47]:
churn_rate = round(cancel_users/total_users*100, 2)

In [48]:
churn_rate

False    37.22
True     34.92
Name: has_kids, dtype: float64